In [2]:
import math
from itertools import product
from collections import defaultdict
import matplotlib as plt
import numpy as np
import shapely
from shapely.geometry import LineString, MultiLineString
from scipy.spatial import distance

In [3]:
test = [2.8533979930661495, 0.8350593062280169, 1.8350735010944388]
lines = [[[(90.92466560343746, 359.8743795603514), (50.64634319592733, 342.689586349763), (141.4154383126297, 0.0), (738.6390963118174, 170.50443049520254)], [(528.746321437764, 592.3512355554849), (250.68498866126174, 548.8781394008547)]], [[(738.6390963118174, 170.50443049520254), (1041.5171545767225, 487.5379093941301), (733.3747470285161, 795.9299552580342), (528.746321437764, 592.3512355554849)]], [[(250.68498866126174, 548.8781394008547), (148.7872773281997, 925.0354834403843), (84.8542414731346, 907.106489722617)], [(84.8542414731346, 907.106489722617), (75.55369137861999, 940.1296991426498), (9.845574646664318, 921.7006750889122), (72.29789360892028, 700.0236357413232)], [(72.29789360892028, 700.0236357413232), (0.0, 679.7554941019043), (90.92466560343746, 359.8743795603514)]]]
#breakpoints = np.asarray([[[-60.535736, 903.22364], [13.366294, 923.257259], [88.824077, 943.712617], [139.973454, 957.578368]], [[-51.029288, 868.142176], [22.872742, 888.175795], [98.330525, 908.631153], [149.479903, 922.496904]], [[0.578007, 677.696767], [74.480037, 697.730385], [149.93782, 718.185743], [201.087197, 732.051494]], [[51.326554, 490.420376], [125.228584, 510.453995], [200.686367, 530.909353], [251.835744, 544.775104]], [[87.017085, 358.712293], [160.919115, 378.745912], [236.376898, 399.20127],[287.526276, 413.067021]]])
breakpoints = np.asarray([[-60.535736, 903.22364], [139.973454, 957.578368], [287.526276, 413.067021], [87.017085, 358.712293]])
#breakpoints = np.asarray([[422.854362, 486.544994], [732.932716, 796.372344], [1048.71745, 480.331781], [738.639096, 170.50443]])


Хуй знает как лучше решить несколько словарей с одинаковыми ключами (то что раньше было args)

In [4]:
class BaseSection:
    def __init__(self, angle, leng, widt, road):
        self.angle = angle
        self.an_normalized = self.angle_normalized()
        self.length, self.width, self.road = leng, widt, road
        self.length_dep = self.length_dependency()
        self.length_r, self.width_r = self.possible_combinations()
        self.flat_length, self.flat_width = self.flat_dictionary()

    def angle_normalized(self):
        if self.angle > math.pi / 4:
            if self.angle < math.pi / 2:
                ang = (math.pi / 2) - self.angle
            elif 3 * math.pi / 4 > self.angle > math.pi / 2:
                ang = self.angle - (math.pi / 2)
            else:
                ang = math.pi - self.angle
        else:
            ang = self.angle
        return ang

    def length_dependency(self):
        length = {}
        for key, value in self.length.items():
            new_w = value[0] + (((value[1] - value[0]) * self.an_normalized) / 0.872665)  # 50 degrees
            length[key] = [round(new_w)]
        return length

    def possible_combinations(self):
        new_length = defaultdict(list)
        new_width = defaultdict(list)
        for key in self.road.keys():
            l = list(product(self.road[key], self.length[key], repeat = 1))
            new_length[key] = [sum(i) for i in l]
            w = list(product(self.road[key], self.width[key], repeat = 1))
            new_width[key] = [sum(i) for i in w]

        return new_length, new_width

    def flat_dictionary(self):
        flat_length = []
        flat_width = []
        for i in self.length_dep.values():
            for v in i:
                flat_length.append(v)
        for h in self.width.values():
            for j in h:
                flat_width.append(j)
        return flat_length, flat_width


In [5]:
class PolyClusters:
    def __init__(self, angle, leng, widt, road, split_points):
        self.angle = angle
        self.split_points = split_points
        self.dist_l, self.dist_w = self.distance_between_points()
        self.basesec = BaseSection(angle, leng, widt, road)
        self.sum_length = self.grid_options(self.basesec.flat_length, self.dist_l, 20)
        self.sum_width = self.grid_options(self.basesec.flat_width, self.dist_w, 0)

    def distance_between_points(self):
        dist_l = round(distance.euclidean(self.split_points[0], self.split_points[1]))
        dist_w = round(distance.euclidean(self.split_points[1], self.split_points[2]))
        return dist_l, dist_w

    def grid_options(self, values, length, k):

        def combination_sum():
            ans = []
            temp = []
            arr = values
            wd = length
            find_numbers(ans, arr, temp, wd, 0, k)
            #a = (np.argmin([val[1] for  val in ans]))
            return ans

        def find_numbers(ans, arr, temp, wd, index,k):
                if k >= wd >= 0:
                    ans.append([list(temp), wd])
                for i in range(index, len(arr)):
                    if (wd - arr[i]) >= 0:
                        temp.append(arr[i])
                        find_numbers(ans, arr, temp, wd - arr[i], i, k)
                        temp.remove(arr[i])

        return combination_sum()

    def point_coordinates(self):
        coor = []
        print (type(self.split_points[1]))
        vec_one = self.split_points[1] - self.split_points[0]
        vec_two = self.split_points[2] - self.split_points[1]
        vec_three = self.split_points[2] - self.split_points[3]
        vec_four = self.split_points[3] - self.split_points[0]
        dist_l = 0
        dist_w = 0
        for i in self.sum_length[1][0]:
            dist_l+=i
            p_one = self.split_points[0] + vec_one / np.linalg.norm(vec_one)*dist_l
            p_three = self.split_points[3] + vec_three / np.linalg.norm(vec_three)*dist_l
            coor.append(np.asarray([p_one, p_three]))
        for i in self.sum_width[4][0]:
            dist_w+=i
            p_two = self.split_points[1] + vec_two / np.linalg.norm(vec_two)*dist_w
            p_four = self.split_points[0] + vec_four / np.linalg.norm(vec_four)*dist_w
            coor.append(np.asarray([p_two, p_four]))
        return np.asarray(coor)



вопрос как обойтись со стандартными параметрами типо ширин проездов и т д?

In [6]:
lengths = {'9':[54,74], '12':[62,89]}
widths = {'9':[72, 90, 108, 126, 144], '12':[65, 81, 98, 114, 130]}
roads = {'9':[68.8, 35.6], '12': [76, 40.6]}
#t = BaseSection(test[2], lengths, widths, roads)
#tes = PolyClusters( test[2], lengths, widths, roads, breakpoints)
t = BaseSection(test[2], lengths, widths, roads)
tes = PolyClusters(test[2], lengths, widths, roads, breakpoints)
a = tes.point_coordinates()
tes.split_points[1] - tes.split_points[0]

<class 'numpy.ndarray'>


array([200.50919 ,  54.354728])

In [7]:
class PoccclyClusters:
    def __init__(self, coordinates, angle, leng, widt, road, split_points):
        self.coordinates = coordinates
        self.angle = angle
        self.split_points = split_points
        self.line = MultiLineString([LineString(c) for c in coordinates])
        self.boundrec = list(self.line.minimum_rotated_rectangle.exterior.coords)
        self.dist_l, self.dist_w = self.distance_between_points()
        self.basesec = BaseSection(angle, leng, widt, road)
        self.sum_length = self.grid_options(self.basesec.flat_length, self.dist_l)
        self.sum_width = self.grid_options(self.basesec.flat_width, self.dist_w)

    def distance_between_points(self):
        dist_l = round(distance.euclidean(self.split_points[0], self.split_points[1]))
        dist_w = round(distance.euclidean(self.split_points[1], self.split_points[2]))
        return dist_l, dist_w

    def grid_options(self, values, length):

        def combination_sum():
            ans = []
            temp = []
            arr = values
            wd = length
            find_numbers(ans, arr, temp, wd, 0)
            a = (np.argmin([val[1] for  val in ans]))
            return ans[a]

        def find_numbers(ans, arr, temp, wd, index):
                if 10 > wd >= 0:
                    ans.append([list(temp), wd])
                for i in range(index, len(arr)):
                    if (wd - arr[i]) >= 0:
                        temp.append(arr[i])
                        find_numbers(ans, arr, temp, wd - arr[i], i)
                        temp.remove(arr[i])

        return combination_sum()

    def point_coordinates(self):
        coor = []
        print (type(self.split_points[1]))
        vec_one = self.split_points[1] - self.split_points[0]
        vec_two = self.split_points[2] - self.split_points[1]
        vec_three = self.split_points[2] - self.split_points[3]
        vec_four = self.split_points[3] - self.split_points[0]
        dist_l = 0
        dist_w = 0
        for i in self.sum_length[0]:
            dist_l+=i
            p_one = self.split_points[0] + vec_one / np.linalg.norm(vec_one)*dist_l
            p_three = self.split_points[3] + vec_three / np.linalg.norm(vec_three)*dist_l
            coor.append(np.asarray([p_one, p_three]))
        for i in self.sum_width[0]:
            dist_w+=i
            p_two = self.split_points[1] + vec_two / np.linalg.norm(vec_two)*dist_w
            p_four = self.split_points[0] + vec_four / np.linalg.norm(vec_four)*dist_w
            coor.append(np.asarray([p_two, p_four]))
        return np.asarray(coor)



In [8]:
def distance_between_pointvvs(self):
        dist_l = []
        dist_w = []
        for b in self.split_points:
            d =[]
            for i, v in enumerate(b):
                if i != len(b) - 1:
                    d.append(round(distance.euclidean(v, b[i+1])))
            dist_l.append(np.asarray(d))
        for b in self.split_points.transpose(1, 0, 2):
            dd =[]
            for i, v in enumerate(b):
                if i != len(b) - 1:
                    dd.append(round(distance.euclidean(v, b[i+1])))
            dist_w.append(np.asarray(dd))
        return np.asarray(dist_l), np.asarray(dist_w)